In [17]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *


%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=2**31-1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
print("Total Params:", FRmodel.count_params())

In [11]:
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [12]:

def triplet_loss(y_true, y_pred, alpha = 0.2):
    """
    Implementation of the triplet loss as defined by formula (3)
    
    Arguments:
    y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    Returns:
    loss -- real number, value of the loss
    """
    
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    
    ### START CODE HERE ### (≈ 4 lines)
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis = -1)
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis = -1)
    # Step 3: subtract the two previous distances and add alpha.
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha) 
    # Step 4: Take the maximum of basic_loss and 0.0. Sum over the training examples.
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0))
    ### END CODE HERE ###
    
    return loss

In [13]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("loss = " + str(loss.eval()))


loss = 528.14294


In [14]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [15]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

array([[ 0.11276132,  0.03207598,  0.1359008 ,  0.11822385,  0.07636503,
         0.13431978,  0.1488092 , -0.1399087 , -0.01258974, -0.06264802,
        -0.06895617,  0.05656989, -0.00625057,  0.00522549,  0.07190286,
        -0.05568055,  0.04310032,  0.04391422, -0.04496381, -0.04642432,
        -0.00843204,  0.12305682, -0.09556346,  0.17989586,  0.01794844,
        -0.1026075 , -0.14342979, -0.0359993 , -0.04165843,  0.13982904,
        -0.04188611,  0.11941338, -0.12171153,  0.05106766,  0.08596219,
        -0.05465616,  0.08568984, -0.06256216,  0.00192187, -0.03967048,
         0.10942454,  0.00864274,  0.06470721, -0.19549364, -0.03102044,
         0.05460496,  0.00480903, -0.02807495, -0.02172411, -0.03455152,
         0.15045404, -0.07591692,  0.1688849 ,  0.01827823,  0.10440759,
         0.02885917, -0.0070692 ,  0.07838199,  0.01713117,  0.01476889,
         0.014934  ,  0.11768681, -0.09003268, -0.18028447,  0.06014771,
         0.08900283,  0.06730763, -0.07218441, -0.1

In [19]:
import numpy as np
import cv2

In [20]:
face_cascade=cv2.CascadeClassifier('/home/sumanth/Desktop/Project/data/haarcascade_frontalface_alt2.xml')
cap = cv2.VideoCapture(0)